# Optimizing Solution Gas-Oil Ratio (GOR) using Ant Colony Optimization (ACO)

In this notebook, I optimized the Glaso correlation to predict the Solution Gas-Oil Ratio (GOR) by using the Ant Colony Optimization (ACO) algorithm. Here’s a breakdown of the steps I followed:

### Step 1: Defined the Objective Function
I began by defining an **objective function** that calculates the Mean Squared Error (MSE) between the predicted GOR (using the Glaso correlation) and the actual observed GOR data. This function served as the error metric that the ACO algorithm minimized.

### Step 2: Implemented the Glaso Correlation
The **Glaso correlation** function was implemented to predict GOR values based on coefficients \(a\), \(b\), and \(c\) alongside temperature (T), pressure (P), gas gravity (\(\gamma_g\)), and API gravity. This correlation is a widely used model to estimate GOR values in reservoir engineering.

### Step 3: Set Up the Ant Colony Optimization (ACO) Algorithm
Using ACO, I set up an optimization process where:
- **Ants** searched for optimal values of coefficients \(a\), \(b\), and \(c\) by exploring solutions guided by pheromones.
- Each **ant** generated candidate solutions for \(a\), \(b\), and \(c\) based on pheromone levels.
- I used **pheromone trails** to reinforce promising solutions, with pheromone levels updated based on the accuracy of each solution.

Key parameters:
- **Alpha (α)** and **Beta (β)** controlled the influence of pheromone levels and heuristic information.
- **Rho (ρ)** determined the pheromone decay rate, and **Q** regulated the pheromone reinforcement strength.

### Step 4: Ran the Optimization
I ran the ACO algorithm for a set number of **iterations** and **ants**:
- The **best coefficients** (\(a\), \(b\), and \(c\)) were updated as the algorithm found lower error values.
- After each iteration, pheromone levels were adjusted based on the solution performance.

### Step 5: Calculated Optimized GOR
After obtaining the best coefficients, I calculated the **optimized GOR values** using the Glaso correlation with the optimized values of \(a\), \(b\), and \(c\). This provided an improved estimate of GOR based on the optimized parameters, enhancing the accuracy of the GOR predictions.

The result was a set of optimized GOR values that minimized the error between predicted and actual observed values.


In [3]:
import numpy as np
import math

# Define the Glaso correlation function for estimating GOR
def glaso_correlation(a, b, c, Pb, Tb, gamma_g, API):
    # Implement Glaso correlation equation with coefficients a, b, c
    Rs_estimated = gamma_g * ((API ** a / (Tb - 460) ** b) * 10 ** (c - (c * np.log10(Pb))))  # Example formula
    return Rs_estimated

# Define the objective function (MSE) for ACO to minimize
def objective_function(a, b, c, Pb_data, Tb_data, GOR_data, gamma_g, API):
    predicted_GOR = np.array([glaso_correlation(a, b, c, Pb, Tb, gamma_g, API) for Pb, Tb in zip(Pb_data, Tb_data)])
    error = np.mean((predicted_GOR - GOR_data) ** 2)
    return error

# ACO function for optimizing parameters a, b, and c
def aco_optimize(Pb_data, Tb_data, GOR_data, gamma_g, API, n_ants=20, n_iterations=100, alpha=1, beta=2, rho=0.1, Q=100):
    # Initialize pheromones
    pheromone_a = np.ones(n_ants)  # Pheromone levels for a
    pheromone_b = np.ones(n_ants)  # Pheromone levels for b
    pheromone_c = np.ones(n_ants)  # Pheromone levels for c
    
    best_a, best_b, best_c = 1.0, 1.0, 1.0
    best_error = float('inf')
    
    # Iteration loop
    for iteration in range(n_iterations):
        solutions_a, solutions_b, solutions_c, errors = [], [], [], []
        
        # Each ant generates a solution
        for ant in range(n_ants):
            # Randomly select values influenced by pheromones
            a = np.random.uniform(0, 10) * pheromone_a[ant] ** alpha
            b = np.random.uniform(0, 5) * pheromone_b[ant] ** beta
            c = np.random.uniform(0, 2) * pheromone_c[ant] ** beta
            
            # Evaluate the solution
            error = objective_function(a, b, c, Pb_data, Tb_data, GOR_data, gamma_g, API)
            
            # Track solutions and errors
            solutions_a.append(a)
            solutions_b.append(b)
            solutions_c.append(c)
            errors.append(error)
            
            # Update best solution if a better error is found
            if error < best_error:
                best_error = error
                best_a, best_b, best_c = a, b, c

        # Pheromone update based on performance
        pheromone_a *= (1 - rho)  # Evaporation
        pheromone_b *= (1 - rho)
        pheromone_c *= (1 - rho)
        
        # Add pheromones based on solution performance
        for ant in range(n_ants):
            pheromone_a[ant] += Q / errors[ant]
            pheromone_b[ant] += Q / errors[ant]
            pheromone_c[ant] += Q / errors[ant]

        # Log best solution for each iteration
        print(f"Iteration {iteration + 1}, Best Error: {best_error}, a: {best_a}, b: {best_b}, c: {best_c}")
    
    return best_a, best_b, best_c, best_error

# Example data
Pb_data = np.array([3000, 3500, 4000, 4500, 5000])  # Reservoir pressure data
Tb_data = np.array([540, 550, 560, 570, 580])       # Temperature data
GOR_data = np.array([400, 420, 430, 460, 480])      # Actual GOR data

# Fixed constants for Glaso model
gamma_g = 0.7   # Gas gravity (example value)
API = 37.0      # API gravity (example value)

# Run ACO optimization
best_a, best_b, best_c, best_error = aco_optimize(Pb_data, Tb_data, GOR_data, gamma_g, API)

print(f"Optimized coefficients: a = {best_a}, b = {best_b}, c = {best_c}")
print(f"Best error: {best_error}")


Iteration 1, Best Error: 70100.49484596922, a: 7.531775177376488, b: 3.913796359602888, c: 0.5502150962682639
Iteration 2, Best Error: 51294.39987391753, a: 6.296975787621814, b: 1.6555263856922033, c: 1.5322138143731854
Iteration 3, Best Error: 51294.39987391753, a: 6.296975787621814, b: 1.6555263856922033, c: 1.5322138143731854
Iteration 4, Best Error: 51294.39987391753, a: 6.296975787621814, b: 1.6555263856922033, c: 1.5322138143731854
Iteration 5, Best Error: 22628.879139142286, a: 4.012487158636705, b: 1.3635125579399752, c: 0.3446886068522175
Iteration 6, Best Error: 22628.879139142286, a: 4.012487158636705, b: 1.3635125579399752, c: 0.3446886068522175
Iteration 7, Best Error: 17833.940014562497, a: 3.9447476920937086, b: 1.0227434471093917, c: 0.5447532093908034
Iteration 8, Best Error: 17833.940014562497, a: 3.9447476920937086, b: 1.0227434471093917, c: 0.5447532093908034
Iteration 9, Best Error: 17833.940014562497, a: 3.9447476920937086, b: 1.0227434471093917, c: 0.54475320939

In [7]:
# Define gamma_g and API based on your data
gamma_g = 0.743  # example value for gas gravity
API = 37  # example value for API gravity

# Assuming the optimized coefficients a, b, c are found
optimized_GOR = np.array([glaso_correlation(best_a, best_b, best_c, Pb, Tb, gamma_g, API) for Pb, Tb in zip(Pb_data, Tb_data)])

print("Optimized GOR values:", optimized_GOR)


Optimized GOR values: [416.61941631 407.30228474 399.21306244 392.08042997 385.71308513]
